# Pangenomics workflow

## Background

This notebook contains the steps to create a pangenome for 9 *Escherichia*/*Shigella* type strains listed in Table 1.  
This workflow uses [Anvi'o](https://anvio.org/) to calculate and visualise the pangenome.  

**Table 1.** Type strains for pangenomics analysis. Shiga-toxin producing *E.coli* in **bold**.  

| Assembly Accession | Organism Name | Total Sequence Length | Assembly Level | Assembly Submission Date |
|------------------|-------------|--------------------:|--------------|----------------------:|
| GCF_000005845.2 | *Escherichia coli* str. K-12 substr. MG1655 | 4641652 | Complete Genome | 2013-09-26 |
| GCF_000006925.2 | *Shigella flexneri* 2a str. 301 | 4828820 | Complete Genome | 2011-08-03 |
| **GCF_000008865.2** | ***Escherichia coli*** **O157:H7 str. Sakai** | **5594605** | **Complete Genome** | **2018-06-08** |
| GCF_002290485.1 | *Shigella boydii* | 4825405 | Contig | 2017-09-12 |
| GCF_013374815.1 | *Shigella sonnei* | 4762774 | Complete Genome | 2020-06-27 |
| GCF_016904755.1 | *Escherichia albertii* | 4631903 | Complete Genome | 2021-03-10 |
| GCF_020097475.1 | *Escherichia fergusonii* | 4645928 | Complete Genome | 2021-09-23 |
| GCF_022354085.1 | *Shigella dysenteriae* | 5192674 | Complete Genome  | 2022-02-22 |
| GCF_902709585.1 | *Escherichia marmotae* | 4584809 | Contig | 2020-07-20 |


## Downloading data

We will use NCBI Datasets command-line tool to download each genome using the assembly accessions from the table above. We only need the annotated genbank files (`gbff`), which we can specify in the command. 

In [ ]:
%%bash

datasets download genome accession \
        GCF_000005845.2 GCF_000006925.2 GCF_000008865.2 \
        GCF_002290485.1 GCF_013374815.1 GCF_016904755.1 \
        GCF_020097475.1 GCF_022354085.1 GCF_902709585.1 \
        --include gbff \
        --no-progressbar

The assemblies are downloaded in one compressed folder, which needs to be unpacked.

In [ ]:
%%bash

unzip ncbi_dataset.zip -d escherichia_shigella

## File formatting

Each annotated assembly file is stored in separatae folder. We need to keep this in mind when giving the paths to each assembly file.  
Next step is to format the genbank files into files that anvi'o needs for calculating the pangenome.  
We'll use a script called `anvi-script-process-genbank` that splits the genbank file into three different files:
 - `*-contigs.fasta`: file with all the contigs
 - `*-gene-calls.txt`: file with coordinates for each gene call in the contigs
 - `*-functions.txt`: functional annotation for each gene call 

As said, the genbank files are in separate folders and they all have the same name (`genomic.gbff`). But the folder name is the accession number. So we need to use the folder name as our genome name. The beginning of the code below first removes the common path to all genbank folders (`escherichia_shigella/ncbi_dataset/data/`) and then removes the redundant file name (`genomic.gbff`). This will leave us with the assembly accession. 

In [ ]:
%%bash

for genome in `ls escherichia_shigella/ncbi_dataset/data/*/genomic.gbff`
do
    name=${genome#escherichia_shigella/ncbi_dataset/data/} 
    name=${name%/genomic.gbff} 
    
    anvi-script-process-genbank \
        -i $genome \
        --output-fasta escherichia_shigella/ncbi_dataset/data/${name}/${name}-contigs.fasta \
        --output-gene-calls escherichia_shigella/ncbi_dataset/data/${name}/${name}-gene-calls.txt \
        --output-functions escherichia_shigella/ncbi_dataset/data/${name}/${name}-functions.txt \
        --annotation-source prodigal \
        --annotation-version v2.6.3
done

## Build the pangenome

After splitting the genbank files we need a file (`fasta.txt`) for anvi'o that has the names of the genomes and the paths for all the files for each of our genomes.  
The code below goes thru all the files produced by `anvi-script-process-genbank` and prints their paths to the `fasta.txt` file. The first line just prints the column headers. 

In [ ]:
%%bash 

echo -e "name\tpath\texternal_gene_calls\tgene_functional_annotation" > fasta.txt
for strain in `ls escherichia_shigella/ncbi_dataset/data/*/*-contigs.fasta`
do
    strain_name=${strain#escherichia_shigella/ncbi_dataset/data/*/}
    echo -e ${strain_name%-contigs.fasta}"\t"$strain"\t"${strain%-contigs.fasta}"-gene-calls.txt\t"${strain%-contigs.fasta}"-functions.txt"
done >> fasta.txt


We will also need a configuration file called `config.json` that has all the options we want to use while building the pangenome.  
You do not have to make this file, it is provided.  

The content of the configuration file is: 
```
{
    "workflow_name": "pangenomics",
    "config_version": "2",
    "max_threads": "8",
    "project_name": "Escherichia_Shigella_pangenome",
    "external_genomes": "external-genomes.txt",
    "fasta_txt": "fasta.txt",
    "anvi_gen_contigs_database": {
        "--project-name": "{group}",
        "--description": "",
        "--skip-gene-calling": "",
        "--ignore-internal-stop-codons": true,
        "--skip-mindful-splitting": "",
        "--contigs-fasta": "",
        "--split-length": "",
        "--kmer-size": "",
        "--skip-predict-frame": "",
        "--prodigal-translation-table": "",
        "threads": ""
    },
    "anvi_pan_genome": {
      "threads": "8"
    }
}
```

Then we have all the files needed to construct the pangenome for our 9 strains. We will use the pangenomics workflow from anvi'o.  
You can read more about anvi'o workflows and the different options from [anvi'o workflow help pages](https://anvio.org/help/main/programs/anvi-run-workflow/).  

In [ ]:
 %%bash
 
 anvi-run-workflow -w pangenomics -c config.json

After this is done, we have only few steps and then we can visualise our pangenome with anvi'o.  
First we import a configuration file that anvi'o calls state, just to have some pre-formatting in our visualisation.  

In [ ]:
%%bash

anvi-import-state -n default -p 03_PAN/Escherichia_Shigella_pangenome-PAN.db -s default_state.json 

Then to make things easier, go to the `03_PAN` folder and print the full path to the folder.  
We will use this path to navigate to the right folder in Visual Studio Code in the last step. So copy it from the notebook. 

In [ ]:
%%bash

cd 03_PAN
pwd

## Visualise the pangenome

Then we are all done and can open **Visual Studio code**.   
Connect to the server and open the folder containing our pangenome files (the path printed above).  
Then run the following command, but remember to change the port to your own port. Each student need to have their own port to be able to visualise the pangenome on their own local browser. 

In [ ]:
%%bash

cd /path/to/course_folder/pangenome_folder

module load anvio
YOUR_PORT=YOUR_PORT_NUMBER

anvi-display-pan -P $YOUR_PORT